In [ ]:
import os
import pandas as pd
import requests
import gradio as gr

In [ ]:
# Sample data input (replace with Power BI data frame input)
df = pd.read_csv("Rationale_Chatbot.csv")

# KEY = "xxxxxxxxx"

In [15]:
MODEL = "llama-3.3-70b-versatile"
API_URL = "https://api.groq.com/openai/v1/chat/completions"

In [16]:
# Function: Build context + send to Groq
def ask_llm(api_key, question):
    # build context from dataframe
    sample_records = df.head(15).to_dict(orient="records")
    context = "\n".join([f"{r['Corp Name']}: {r['Rationale']}" for r in sample_records])
# ------------------------------------------------------------------------------------------------
    prompt = f"""
You are a helpful assistant analyzing audit flag rationales for a government auditing system.
Here are sample flagged companies and the reasons:

{context}

Question: {question}
Answer:
"""
# ------------------------------------------------------------------------------------------------
    # Prepare and send the request
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "model": MODEL,
        "messages": [
            {"role": "system", "content": "You are an audit expert helping explain flags."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.3
    }

    response = requests.post(API_URL, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        return f"Error: {response.status_code} - {response.text}"


In [17]:
# Build the Gradio interface with two inputs: API key and question
interface = gr.Interface(
    fn=ask_llm,
    inputs=[
        gr.Textbox(label="Groq API Key", type="password", placeholder="Enter your Groq API Key"),
        gr.Textbox(label="Question", lines=2, placeholder="e.g., Why was Minty Co flagged?")
    ],
    outputs=gr.Textbox(label="LLM Explanation"),
    title="Audit Flag Q&A",
    description="Enter your Groq API Key and a question to get an explanation of flagged audit records."
)


In [ ]:
# KEY = "xxxxxxxxx"
interface.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
